In [18]:
from imputationLibrary import forwardFilling, hotDeck, meanImputation, movingAverage, splineInterpolation, separate, randomSampleImputation
from imputationLibrary.util import util
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from tsmoothie.smoother import *
import matplotlib.pyplot as plt

In [30]:
df = pd.read_csv(r'output\/CompleteDengue.csv',index_col=0, parse_dates=True)
df = df.sort_index()
df_train = df.loc['1990-04-30':'2004-06-03'].dropna(how='all', axis=1) #Selecting training data
df_test = df.loc['2004-06-10':, df_train.columns] #Selecting test data
period=7

In [31]:
for i in range(0,30):
    df_white_noise, df_seasonal, df_trended, df_trend_and_seasonal = separate.separate(df_train, period)
    df_seasonal_train, df_seasonal_test = randomSampleImputation.input(df_train.loc[:, df_seasonal.columns], df_test.loc[:, df_seasonal.columns], sliding_window_size = 0.03)
    if df_white_noise.columns.any():
        df_white_noise_train, df_white_noise_test =  meanImputation.input(df_train.loc[:, df_white_noise.columns], df_test.loc[:, df_white_noise.columns])
    df_trended_train, df_trended_test =  forwardFilling.input(df_train.loc[:, df_trended.columns], df_test.loc[:, df_trended.columns])
    df_trend_and_seasonal_train, df_trend_and_seasonal_test =  splineInterpolation.input(df_train.loc[:, df_trend_and_seasonal.columns], df_test.loc[:, df_trend_and_seasonal.columns])
    df_train = pd.concat([df_seasonal_train, df_white_noise_train, df_trended_train, df_trend_and_seasonal_train], axis=1, join='outer')
    df_test = pd.concat([df_seasonal_test, df_white_noise_test, df_trended_test, df_trend_and_seasonal_test], axis=1, join='outer')
    ### USE KALMAN FILTER TO SMOOTH ALL DATA (ONLY VISUALIZATION PURPOSE) ###

    smoother_train = KalmanSmoother(component='level_longseason', 
                             component_noise={'level':0.1, 'longseason':0.1}, 
                              n_longseasons=365)
    smoother_train.smooth(df_train.T)

    smoother_test = KalmanSmoother(component='level_longseason', 
                              component_noise={'level':0.1, 'longseason':0.1}, 
                              n_longseasons=365)
    smoother_test.smooth(df_test.T)
    df_train = pd.DataFrame(data = smoother_train.smooth_data.T, index = df_train.index, columns= df_train.columns)
    df_test = pd.DataFrame(data = smoother_test.smooth_data.T, index = df_test.index, columns= df_test.columns)

    df_train.to_csv(r'output\/CompleteDengueNatureKalmanTrain_'+str(i)+'.csv', index = True)
    df_test.loc['2004-06-10':, df_train.columns].to_csv(r'output\/CompleteDengueNatureKalmanTest_'+str(i)+'.csv', index = True)

Could not determine nature of the feature sj_ndvi_nw and it will be assigned to a group randomly.


C:\Users\silva\Anaconda3\envs\mestrado\lib\site-packages\numpy\core\fromnumeric.py:58: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


Could not determine nature of the feature iq_reanalysis_tdtr_k and it will be assigned to a group randomly.
Could not determine nature of the feature iq_station_precip_mm and it will be assigned to a group randomly.
Could not determine nature of the feature iq_ndvi_se and it will be assigned to a group randomly.


TypeError: Index(...) must be called with a collection of some kind, 'seasonal' was passed

In [32]:
df_seasonal.columns

Index(['sj_ndvi_ne', 'sj_ndvi_sw', 'iq_ndvi_nw', 'iq_ndvi_se', 'iq_ndvi_sw',
       'sj_ndvi_se', 'sj_ndvi_nw', 'iq_reanalysis_tdtr_k'],
      dtype='object')

In [33]:
df_trended.columns

Index(['iq_precipitation_amt_mm', 'iq_reanalysis_precip_amt_kg_per_m2',
       'iq_reanalysis_sat_precip_amt_mm', 'iq_station_avg_temp_c',
       'iq_station_diur_temp_rng_c'],
      dtype='object')

In [34]:
df_trend_and_seasonal.columns

Index(['iq_ndvi_ne', 'sj_reanalysis_air_temp_k', 'sj_reanalysis_avg_temp_k',
       'sj_reanalysis_dew_point_temp_k', 'sj_reanalysis_max_air_temp_k',
       'sj_reanalysis_min_air_temp_k',
       'sj_reanalysis_relative_humidity_percent', 'sj_station_precip_mm',
       'iq_reanalysis_air_temp_k', 'iq_reanalysis_avg_temp_k',
       'iq_reanalysis_dew_point_temp_k', 'iq_reanalysis_max_air_temp_k',
       'iq_reanalysis_min_air_temp_k',
       'iq_reanalysis_relative_humidity_percent',
       'iq_reanalysis_specific_humidity_g_per_kg', 'iq_station_max_temp_c',
       'iq_station_min_temp_c', 'sj_precipitation_amt_mm',
       'sj_reanalysis_precip_amt_kg_per_m2', 'sj_reanalysis_sat_precip_amt_mm',
       'sj_reanalysis_specific_humidity_g_per_kg', 'sj_reanalysis_tdtr_k',
       'sj_station_avg_temp_c', 'sj_station_diur_temp_rng_c',
       'sj_station_max_temp_c', 'sj_station_min_temp_c',
       'iq_station_precip_mm'],
      dtype='object')

In [35]:
df_white_noise.columns

Index([], dtype='object')

In [36]:
if df_white_noise.columns.any():
    print("oi")